<a href="https://colab.research.google.com/github/alvinsbkt/sentiment-analysis-rotten-tomatoes/blob/main/Sentiment_Analysis_on_Rotten_Tomatoes_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.tokenize import TweetTokenizer
import datetime
#import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score
#from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
pd.set_option('max_colwidth',400)

# Reading the input files

In [ ]:
train = pd.read_csv('/content/drive/My Drive/WebMining/train.tsv', sep="\t")


In [ ]:
train.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what is good for the goose,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is good for the goose,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for the goose,2


In [ ]:
train.loc[train.SentenceId == 2]

,PhraseId,SentenceId,Phrase,Sentiment
63,64,2,"This quiet , introspective and entertaining independent is worth seeking .",4
64,65,2,"This quiet , introspective and entertaining independent",3
65,66,2,This,2
66,67,2,"quiet , introspective and entertaining independent",4
67,68,2,"quiet , introspective and entertaining",3
68,69,2,quiet,2
69,70,2,", introspective and entertaining",3
70,71,2,introspective and entertaining,3
71,72,2,introspective and,3
72,73,2,introspective,2


In [ ]:
train['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

# Cleaning the data

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [ ]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
train['clean_review']=clean_review(train.Phrase.values)

In [ ]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(newdf['cleaning'])
sequences = tokenizer.texts_to_sequences(newdf['cleaning'])
print('Found %s unique tokens.' %len(tokenizer.word_index))

In [ ]:
data = keras.preprocessing.sequence.pad_sequences(sequences)
labels = np.array(pd.get_dummies((newdf['sentiment'])))
print('Shape of review tensor:', data.shape)
print('shape of sentiment tensor:', labels.shape)

# Balancing the data by Resampling

In [ ]:
from sklearn.utils import resample
train_2 = train[train['Sentiment']==2]
train_1 = train[train['Sentiment']==1]
train_3 = train[train['Sentiment']==3]
train_4 = train[train['Sentiment']==4]
train_5 = train[train['Sentiment']==0]
train_2_sample = resample(train_2,replace=True,n_samples=75000,random_state=123)
train_1_sample = resample(train_1,replace=True,n_samples=75000,random_state=123)
train_3_sample = resample(train_3,replace=True,n_samples=75000,random_state=123)
train_4_sample = resample(train_4,replace=True,n_samples=75000,random_state=123)
train_5_sample = resample(train_5,replace=True,n_samples=75000,random_state=123)

df_upsampled = pd.concat([train_2, train_1_sample,train_3_sample,train_4_sample,train_5_sample])

In [ ]:
df_upsampled.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series
5,6,1,of escapades demonstrating the adage that what is good for the goose,2,of escapade demonstrating the adage that what is good for the goose


# Data Processing for ML

In [ ]:
text = ' '.join(df_upsampled.loc[df_upsampled.Sentiment == 4, 'Phrase'].values)
text_trigrams = [i for i in ngrams(text.split(), 3)]

In [ ]:
Counter(text_trigrams).most_common(30)

[(('one', 'of', 'the'), 1644),
 (('of', 'the', 'year'), 832),
 (('of', 'the', 'best'), 677),
 (('of', 'the', 'most'), 612),
 (('is', 'one', 'of'), 407),
 (('One', 'of', 'the'), 370),
 ((',', 'and', 'the'), 333),
 (('the', 'year', "'s"), 326),
 (('It', "'s", 'a'), 323),
 (('the', 'edge', 'of'), 300),
 (('it', "'s", 'a'), 299),
 (('a', 'movie', 'that'), 297),
 (('of', 'your', 'seat'), 273),
 (('the', 'film', 'is'), 267),
 (('the', 'kind', 'of'), 267),
 (('.', 'is', 'a'), 264),
 (('the', 'film', "'s"), 264),
 (('as', 'one', 'of'), 254),
 ((',', 'the', 'film'), 253),
 (('edge', 'of', 'your'), 249),
 ((',', 'this', 'is'), 236),
 (('as', 'well', 'as'), 231),
 ((',', 'it', "'s"), 226),
 (('film', 'that', 'is'), 223),
 (('.', 'It', "'s"), 218),
 (('a', 'film', 'that'), 211),
 ((',', 'funny', ','), 208),
 (('some', 'of', 'the'), 206),
 (('year', "'s", 'best'), 188),
 (('a', 'solid', 'cast'), 178)]

In [ ]:
tokenizer = TweetTokenizer()

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)
full_text = list(df_upsampled['clean_review'].values)
vectorizer.fit(full_text)
df_upsampled_vectorized = vectorizer.transform(df_upsampled['clean_review'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
y = df_upsampled['Sentiment']

# Applying ML algorithm

In [ ]:
from keras.utils import to_categorical
X = df_upsampled['clean_review']
#test_set = test['clean review']
#Y = train['Sentiment']
Y = to_categorical(df_upsampled['Sentiment'].values)
print(Y)

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


# splitting training set into training and validation set

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=123)

In [ ]:
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)

(284686,) (284686, 5)
(94896,) (94896, 5)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Total number of words/features

In [ ]:
all_words=' '.join(X_train)
all_words=word_tokenize(all_words)
#print(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word
#X_train.head()

13728

# Number of words for each phrase/text

In [ ]:
r_len=[]
for text in X_train:
    word=word_tokenize(text)
  #  print(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
max_words = MAX_REVIEW_LEN
max_words

48

In [ ]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 30
num_classes=5

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

# Tokenizing the words

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

# Sequence Padding

In [ ]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
#print(X_train.shape,X_val.shape)

In [ ]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt

#LSTM

In [ ]:
# importing required packages
import re
import csv
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import callbacks
from keras.preprocessing import sequence
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
model1=Sequential()
model1.add(Embedding(max_features,100))
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Bidirectional(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)))
model1.add(Bidirectional(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False)))
model1.add(Dense(num_classes,activation='softmax'))


model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1372800   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         49664     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 1,473,637
Trainable params: 1,473,637
Non-trainable params: 0
____________________________________________

# Fitting the model

In [ ]:
#%%time
model1.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=epochs, batch_size=batch_size, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 284686 samples, validate on 94896 samples
Epoch 1/10
284686/284686 [==============================] - 348s 1ms/step - loss: 0.9493 - accuracy: 0.6030 - val_loss: 0.7616 - val_accuracy: 0.6907
Epoch 2/10
284686/284686 [==============================] - 346s 1ms/step - loss: 0.7191 - accuracy: 0.7121 - val_loss: 0.6697 - val_accuracy: 0.7344
Epoch 3/10
284686/284686 [==============================] - 347s 1ms/step - loss: 0.6370 - accuracy: 0.7488 - val_loss: 0.6264 - val_accuracy: 0.7566
Epoch 4/10
284686/284686 [==============================] - 344s 1ms/step - loss: 0.5857 - accuracy: 0.7726 - val_loss: 0.5996 - val_accuracy: 0.7698
Epoch 5/10
284686/284686 [==============================] - 338s 1ms/step - loss: 0.5491 - accuracy: 0.7874 - val_loss: 0.5707 - val_accuracy: 0.7843
Epoch 6/10
284686/284686 [==============================] - 343s 1ms/step - loss: 0.5228 - accuracy: 0.8003 - val_loss: 0.5602 - val_accuracy: 0.7881
Epoch 7/10
284686/284686 [=======================

In [ ]:
Y_val = [np.argmax(y, axis=None, out=None) for y in Y_val]
Y_val[:10]

[1, 3, 2, 1, 4, 1, 4, 1, 1, 0]

In [ ]:
from sklearn import metrics
y_pred=model1.predict_classes(X_val)

print('Akurasi:',metrics.accuracy_score(y_pred,Y_val))

Akurasi: 0.808706373292868


# Predicting the model

## CNN model

In [ ]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

In [ ]:
model2 = Sequential()

# Input / Embdedding
model2.add(Embedding(max_features, 150, input_length=max_words))

# CNN
model2.add(SpatialDropout1D(0.2))

model2.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Flatten())

# Output layer
model2.add(Dense(5, activation='sigmoid'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 284686 samples, validate on 94896 samples
Epoch 1/3
284686/284686 [==============================] - 138s 483us/step - loss: 0.8830 - acc: 0.6329 - val_loss: 0.7086 - val_acc: 0.7145
Epoch 2/3
284686/284686 [==============================] - 138s 486us/step - loss: 0.6148 - acc: 0.7567 - val_loss: 0.5950 - val_acc: 0.7685
Epoch 3/3
284686/284686 [==============================] - 139s 487us/step - loss: 0.5113 - acc: 0.8017 - val_loss: 0.5358 - val_acc: 0.7969


In [ ]:
pred2=model2.predict_classes(X_test,verbose=1)
sub.Sentiment=pred2
sub.to_csv('sub2.csv',index=False)
#sub.head()

66292/66292 [==============================] - 6s 84us/step


# CNN+BiLSTM


In [ ]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(Bidirectional(LSTM(128,return_sequences=True)))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1372800   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 24, 256)           197632    
_________________________________________________________________
dropout_8 (Dropout)          (None, 24, 256)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6144)             

In [ ]:
%%time
model3.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=epochs, batch_size=batch_size, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 284686 samples, validate on 94896 samples
Epoch 1/30
284686/284686 [==============================] - 589s 2ms/step - loss: 0.8842 - accuracy: 0.6360 - val_loss: 0.6996 - val_accuracy: 0.7137
Epoch 2/30
284686/284686 [==============================] - 589s 2ms/step - loss: 0.6458 - accuracy: 0.7438 - val_loss: 0.6301 - val_accuracy: 0.7461
Epoch 3/30
284686/284686 [==============================] - 591s 2ms/step - loss: 0.5644 - accuracy: 0.7792 - val_loss: 0.5520 - val_accuracy: 0.7866
Epoch 4/30
284686/284686 [==============================] - 592s 2ms/step - loss: 0.5133 - accuracy: 0.8024 - val_loss: 0.5238 - val_accuracy: 0.8019
Epoch 5/30
284686/284686 [==============================] - 587s 2ms/step - loss: 0.4770 - accuracy: 0.8173 - val_loss: 0.5047 - val_accuracy: 0.8106
Epoch 6/30
284686/284686 [==============================] - 587s 2ms/step - loss: 0.4484 - accuracy: 0.8287 - val_loss: 0.5047 - val_accuracy: 0.8162
Epoch 7/30
284686/284686 [=======================

In [ ]:
Y_val = [np.argmax(y, axis=None, out=None) for y in Y_val]
Y_val[:10]

[1, 3, 2, 1, 4, 1, 4, 1, 1, 0]

In [ ]:
from sklearn import metrics
y_pred=model3.predict_classes(X_val)

print('Akurasi:',metrics.accuracy_score(y_pred,Y_val))

Akurasi: 0.8658742201989547
